In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb && make pipeline`
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_1803/964790820.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106708,170414,https://www.slowboring.com/p/is-this-mailbag-s...,"Is this mailbag ""screen time""?",\N,\N,2023-12-01 17:55:16.942031+00,2023-12-01 17:55:16.942031+00,NaN,\N,Yglesias,43.0,0.0,\N
106709,170415,https://www.askamanager.org/2023/12/its-your-f...,it’s your Friday good news,\N,www.askamanager.org,2023-12-01 17:55:16.943301+00,2023-12-01 19:03:59.589951+00,0.0,\N,AskManager,80.0,0.0,\N
106710,170416,https://thedispatch.com/newsletter/morning/pax...,Pax Kissinger,\N,\N,2023-12-01 17:55:16.94453+00,2023-12-01 17:55:16.94453+00,NaN,\N,Dispatch,51.0,0.0,\N
106711,170417,https://tellingthefuture.substack.com/p/michae...,Michael Story on Making Useful Forecasts,\N,\N,2023-12-01 17:55:16.945717+00,2023-12-01 17:55:16.945717+00,NaN,\N,DeNeufville,93.0,0.0,\N


In [2]:
links['added'].max()

'2023-12-01 17:55:16.946876+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
106670    170374
106671    170375
106672    170377
106673    170378
106674    170379
Name: id, Length: 14495, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/36
...11/36
...21/36
...31/36
...36/36


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106743,170449,https://docs.google.com/document/d/1lwj_pA-p8g...,Prep for Athens Roundtable - Google Docs,\N,\N,2023-12-01 14:36:14.574463,2023-12-01 14:36:14.574467,\N,\N,Custom,100.0,0.0,\N
106744,170450,https://www.facebook.com/jueyan?notif_id=17013...,Facebook,\N,\N,2023-12-01 14:36:14.580015,2023-12-01 14:36:14.580019,\N,\N,Custom,2.0,0.0,\N
106745,170451,https://lucid.app/lucidchart/62cba5bb-1fdb-443...,Blank diagram: Lucidchart,\N,\N,2023-12-01 14:36:14.585569,2023-12-01 14:36:14.585574,\N,\N,Custom,37.0,0.0,\N
106746,170452,https://www.google.com/search?q=yoshua+bengio+...,yoshua bengio commission report - Google Search,\N,\N,2023-12-01 14:36:14.591841,2023-12-01 14:36:14.591845,\N,\N,Custom,63.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106743,170449,https://docs.google.com/document/d/1lwj_pA-p8g...,Prep for Athens Roundtable - Google Docs,\N,\N,2023-12-01 14:36:14.574463,2023-12-01 14:36:14.574467,\N,\N,Custom,100.0,0.0,\N
106744,170450,https://www.facebook.com/jueyan?notif_id=17013...,Facebook,\N,\N,2023-12-01 14:36:14.580015,2023-12-01 14:36:14.580019,\N,\N,Custom,2.0,0.0,\N
106745,170451,https://lucid.app/lucidchart/62cba5bb-1fdb-443...,Blank diagram: Lucidchart,\N,\N,2023-12-01 14:36:14.585569,2023-12-01 14:36:14.585574,\N,\N,Custom,37.0,0.0,\N
106746,170452,https://www.google.com/search?q=yoshua+bengio+...,yoshua bengio commission report - Google Search,\N,\N,2023-12-01 14:36:14.591841,2023-12-01 14:36:14.591845,\N,\N,Custom,63.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106743,170449,https://docs.google.com/document/d/1lwj_pA-p8g...,Prep for Athens Roundtable - Google Docs,\N,\N,2023-12-01 14:36:14.574463,2023-12-01 14:36:14.574467,\N,\N,Custom,100,0,\N
106744,170450,https://www.facebook.com/jueyan?notif_id=17013...,Facebook,\N,\N,2023-12-01 14:36:14.580015,2023-12-01 14:36:14.580019,\N,\N,Custom,2,0,\N
106745,170451,https://lucid.app/lucidchart/62cba5bb-1fdb-443...,Blank diagram: Lucidchart,\N,\N,2023-12-01 14:36:14.585569,2023-12-01 14:36:14.585574,\N,\N,Custom,37,0,\N
106746,170452,https://www.google.com/search?q=yoshua+bengio+...,yoshua bengio commission report - Google Search,\N,\N,2023-12-01 14:36:14.591841,2023-12-01 14:36:14.591845,\N,\N,Custom,63,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)